<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_cassandra_issues_jira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Dependencies

In [11]:
print('Install dependencies...')
print('Update apt. ')
!sudo apt update > apt_update.log
print('Install software-properties-common.')
!sudo apt install software-properties-common > spc.log
print('Add ppa-git-core repository.')
!sudo add-apt-repository -y ppa:git-core/ppa > ppa_git_core.log
print('Install the latest version of git.')
!sudo apt install -y git > install_git.log
!git --version > git_version.log
print('Install pydriller')
!pip3 install pydriller > install_pydriller.log
!pip3 list | grep PyDriller > versao_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install dependencies...
Update apt. 


Install software-properties-common.


Add ppa-git-core repository.
Install the latest version of git.


Install pydriller
Install gitpython.
Install Jira Python lib.
All depenpencies installed!
cat: install.log: input file is output file
Details in install.log


In [12]:
from jira import JIRA
import pandas as pd
import os
import subprocess
import requests
import threading
import datetime
from git import Repo
import configparser # para manipular arquivos .conf
import pydriller

def get_repository_name(url):
  repository_name = os.path.splitext(os.path.basename(url))[0]
  return repository_name

class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        try:
          for i in range(self.qtd_repositorios):
              nome = get_repository_name(url=self.lista_urls[i])
              tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
              tdi.name = 'Thread-'+str(i+1)
              inicio_thread = datetime.datetime.now()
              print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
              self.lista_threads.append(tdi)
              elemento = (nome, inicio_thread)
              self.lista_tempo_inicio.append(elemento)
              tdi.start()

          for each in self.lista_threads:
              each.join()
        except Exception as ex:
          print(f'Erro no clona_repositorio: {str(ex)}')

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de clonagem do {t1[0]} é: {t2[1]-t1[1]} s")

# extrai o issue tracker salvo na mensagem de cada commit
def extract_cassandra_issue(text):
  """
  Extract the CASSANDRA-XYZ issue from the text.

  Args:
    text: The text to extract the issue from.

  Returns:
    The issue number, or None if the issue cannot be found.
  """

  issues_number = []

  lines = text.split('\n')
  for line in lines:
    if 'CASSANDRA-' in line :
      issue_number = line.split('-')[1]
      issue_number = 'CASSANDRA-' + issue_number
      issues_number.append(issue_number)
  return issues_number

# Commit Analysis

In [13]:
link_repositorio = 'https://github.com/apache/cassandra.git'

# Clona e repositorio
lista_repositorios = [link_repositorio]
qtd_repositorios = len(lista_repositorios)
cloner = RepoCloner(qtd_repositorios=len(lista_repositorios), lista_urls=lista_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()
print(f'Clonagem de todos os {qtd_repositorios} concluída com sucesso!')

Thread 1 iniciada em 2023-09-13 15:00:06.518095 - clonando cassandra:https://github.com/apache/cassandra.git...
Erro ao clonar o repo: https://github.com/apache/cassandra.git - Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/apache/cassandra.git cassandra
  stderr: 'fatal: destination path 'cassandra' already exists and is not an empty directory.
'
Clonagem de todos os 1 concluída com sucesso!


## Faixa de commits analisados: 3.0.0 a 3.11.11

In [14]:
# 3. Define a faixa de commits que será analisada
# 3.1 Mostra todos os commits entre as tags 3.0.0 e 3.11.11
!cd cassandra && git log --pretty="%H;%ai;%s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11-full.txt
!cd cassandra && git log --pretty="%H %s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11-msg.txt
!echo "10 primeiros commits entre as versões cassandra-3.0.0...cassandra-3.11.11"
!cd cassandra && head commits-3.0.0-3.11.11-msg.txt
!echo "..."

# Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11
!echo "Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11: "
!cd cassandra && cat commits-3.0.0-3.11.11-msg.txt | wc -l

10 primeiros commits entre as versões cassandra-3.0.0...cassandra-3.11.11
4cafe2288e56e1135d65e76adbcd6c2de9306d6b Prepare for release 3.11.11
4950878639056261746e24b2a5de29d739668e2d update IDEA setup generation to mark test/conf as test resources
8857d0448914c2b8bb3e4e8f5f49b1d369f53893 Fix broken IntelliJ configuration Authored by Jacek Lewandowski; reviewed by Ekaterina Dimitrova and Michael Semb Wever for CASSANDRA-16618
3e54ed76b29b752f2c17baa98901adf47e5897d1 Don't put apidocs (javadoc) into the binary artifact
7fafda8e9063212168487ae98aad044837b7e2ab Don't fork jvms in the build
b2cfe2e5d7662c4d5900483d1838c718740ea13c Fix warning about multiple versions of ant detected when running ant test patch by Ekaterina Dimitrova; reviewed by Brandon Williams and Mick Semb Wever for CASSANDRA-16571
fd7732389aa77a94d3d5879165e8acde48125d5b Update IDE, Eclipse and NetBeans, classpaths for tests
c44dfab9b5b62820d70aba062ba3c9b9117912db Add JStackJUnitTask to avoid downloading the jar
e69505

### Lista de commits entre 3.0.0 e 3.11.11

In [15]:
# Lista os commits entre duas tags
!cd cassandra && git log --pretty="%H" cassandra-3.0.0...cassandra-3.11.11 > commmitstags300to31111.txt
commits_tag_3_from_000_to_111111 = !cd cassandra && cat commmitstags300to31111.txt
print(f'Qtd: { len(commits_tag_3_from_000_to_111111) }, {commits_tag_3_from_000_to_111111}')

Qtd: 5559, ['...']


In [16]:
#!cd cassandra && git show a37f8becf2d5df2895febbd8ec0cbd8ba793be8f
#!cd cassandra && git show 4cafe2288e56e1135d65e76adbcd6c2de9306d6b
# git log --format=%B -n 1 <commit>
!cd cassandra && git log --format=%B -n 1 093454205498bad7094cb8fb362d60e95296b092

Only log yaml config once, at startup

patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217



In [17]:
commit_inicial = commits_tag_3_from_000_to_111111[-1]
commit_final = commits_tag_3_from_000_to_111111[0]
data_commit_inicial = '03/08/2015'
data_commit_final = '25/06/2021'
lista_commits_analisados = commits_tag_3_from_000_to_111111

In [18]:
import tqdm
import pydriller

total_commits = len(lista_commits_analisados)
dict_commit_modified_files = {}
i = 0

iterable_commits = tqdm.tqdm(pydriller.Repository("cassandra", only_commits=lista_commits_analisados).traverse_commits(), total=total_commits)

for commit in iterable_commits:
  list_modified_files = []
  for m in commit.modified_files:
    list_modified_files.append(m.filename)
  qtd_modified_files = len(list_modified_files)
  commit_message = commit.msg
  elemento = (commit_message, list_modified_files, qtd_modified_files)
  dict_commit_modified_files[commit.hash] = elemento
  percentage = (i + 1) / total_commits * 100
  iterable_commits.set_description(f"Progress Commit Analysis")
  i += 1

Progress Commit Analysis: 100%|██████████| 5559/5559 [01:18<00:00, 70.52it/s]


In [19]:
qtd_total_modified_files = 0
for e in dict_commit_modified_files.values():
  qtd_total_modified_files = qtd_total_modified_files + e[2]

In [20]:
print(f'Commit Inicial: {commit_inicial} - {data_commit_inicial}')
print(f'Commit Final: {commit_final} - {data_commit_final}')
print(f'Qtd de commits analisados: {len(lista_commits_analisados)}')
print(f'Qtd de arquivos modificados: {qtd_total_modified_files}')

Commit Inicial: a37f8becf2d5df2895febbd8ec0cbd8ba793be8f - 03/08/2015
Commit Final: 4cafe2288e56e1135d65e76adbcd6c2de9306d6b - 25/06/2021
Qtd de commits analisados: 5559
Qtd de arquivos modificados: 12254


# Dicionário commit e mensagem

### Lista de Issue Id dos commits da faixa analisada (3.0.0 a 3.11.11)

Referente aos commits que tiveram o id do issue salvo na mensagem de commit

In [22]:
with open('/content/cassandra/commits-3.0.0-3.11.11-msg.txt', mode='r') as f:
  conteudo = f.read()

list_issue_number = extract_cassandra_issue(conteudo)
total_items = len(list_issue_number)
i = 0

iterable_list_issue_number = tqdm.tqdm(list_issue_number, total=total_items)
# Faz a limpeza para garantir o id do Issue tracker
list_cassandra_issues_analyzed = []
for each in iterable_list_issue_number:
  issue = each.split()
  if len(issue) == 1:
    elemento = issue[0].replace('"', '')
    elemento = elemento.replace(")", "")
    elemento = elemento.replace(".", "")
    list_cassandra_issues_analyzed.append(elemento)
  percentage = (i + 1) / total_items * 100
  iterable_list_issue_number.set_description(f"Progress Message Analysis")
  i += 1

Progress Message Analysis: 100%|██████████| 177/177 [00:00<00:00, 674.88it/s]


In [23]:
print(f'{len(list_cassandra_issues_analyzed)} commits com Id de issue: {list_cassandra_issues_analyzed}')

100 commits com Id de issue: ['CASSANDRA-16618', 'CASSANDRA-16571', 'CASSANDRA-16618', 'CASSANDRA-15985', 'CASSANDRA-16736', 'CASSANDRA-16804', 'CASSANDRA-16714', 'CASSANDRA-16695', 'CASSANDRA-16690', 'CASSANDRA-16654', 'CASSANDRA-16633', 'CASSANDRA-16628', 'CASSANDRA-16495', 'CASSANDRA-13853', 'CASSANDRA-15897', 'CASSANDRA-16478', 'CASSANDRA-16478', 'CASSANDRA-16478', 'CASSANDRA-16146', 'CASSANDRA-15789', 'CASSANDRA-16152', 'CASSANDRA-15907', 'CASSANDRA-15814', 'CASSANDRA-14993', 'CASSANDRA-15012', 'CASSANDRA-14660', 'CASSANDRA-14660', 'CASSANDRA-13797', 'CASSANDRA-11500', 'CASSANDRA-13004', 'CASSANDRA-13559', 'CASSANDRA-12847', 'CASSANDRA-13412', 'CASSANDRA-13412', 'CASSANDRA-13228', 'CASSANDRA-12835', 'CASSANDRA-13276', 'CASSANDRA-13276', 'CASSANDRA-13307', 'CASSANDRA-13038', 'CASSANDRA-13090', 'CASSANDRA-13090', 'CASSANDRA-13090', 'CASSANDRA-13046', 'CASSANDRA-12348', 'CASSANDRA-12868', 'CASSANDRA-11743', 'CASSANDRA-12642', 'CASSANDRA-12499', 'CASSANDRA-12499', 'CASSANDRA-11844', '

In [ ]:
for k,v in dict_commit_modified_files.items():
  if v[0] is not None:
    print(k, v[0])

In [ ]:
dict_message_commits_valido = {}

for k,v in dict_commit_modified_files.items():
  if v[0] is not None:
    for cassandra_issue in list_cassandra_issues_analyzed:
      if cassandra_issue in v[0]:
        print(k, cassandra_issue, v[0])
        dict_message_commits_valido[k] = cassandra_issue
        break

In [32]:
for k, v in dict_message_commits_valido.items():
  print(k,v)

47341eb6aaca318d0ffc0e9f906b98db50b9e9ff CASSANDRA-9945
1db386e46e451250b2375060f92093f670091c02 CASSANDRA-9241
557bbbccb0eddc9f2ba6431b023b3ded253de056 CASSANDRA-7392
99617a529378f00cb86ab733959c7be9966860c9 CASSANDRA-10280
851aed7ee78684dd1de03d5c3771117840c8e0f0 CASSANDRA-10280
b5240204d7aa2a32c6649d19da2b961c856cde28 CASSANDRA-10679
425dc5f7dc9802b25f933d762d42ea99a215617c CASSANDRA-8142
9135340a13531930f26fb8175b057342791ff044 CASSANDRA-9628
94c361d1a7a63490f1b66f5b10e02579516c5eae CASSANDRA-9748
ce7f5cc332a3f5f8c49a4bc22dbb33bddb813a3d CASSANDRA-10859
ebbd516985bc3e2859ae00e63a024b837cb4b429 CASSANDRA-10814
e43883137750d62d8bafb3058d5ee4e322221733 CASSANDRA-10701
4e5867affc6c02b4a84ecee5e82e2ac15c2e3a1c CASSANDRA-10921
cae395026af3f82afbbee6d2ab090f985ee006d3 CASSANDRA-10708
867fd8e2e9c33e8f2ee1d13c3b7dd177434ec645 CASSANDRA-10917
8bafc180b62f9bb60ac50a1d87e510bfa9952fcf CASSANDRA-10918
6bd0f7ac1037f7171291e2a8a92d9b013b1a5626 CASSANDRA-10941
21191e6efa8a31292791717295f460faacec5

In [33]:
len(dict_message_commits_valido)

149

## Critical Files found by ATDCodeAnalyzer

In [34]:
lista_arquivos_criticos = ['StorageService', 'ColumnFamilyStore', 'DatabaseDescriptor', 'CompactionManager', 'StorageProxy', 'SSTableReader', 'Config', 'CassandraDaemon', 'SelectStatement', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService']
len(lista_arquivos_criticos)

12

# Access Cassandra Jira Issues

In [35]:
'''
Create a file credentials.conf with the following content:
[credentials]
username = ?
password = ?
'''
config = configparser.ConfigParser()
config.read('credentials.conf')

username = config['credentials']['username']
password = config['credentials']['password']

SERVIDOR_JIRA = 'https://issues.apache.org/jira'

try:
  # Create a JIRA client instance
  jira = JIRA(
      server=SERVIDOR_JIRA,  # Replace with the Cassandra JIRA server URL
      basic_auth=(username, password),  # Replace with your JIRA credentials
  )
  print(f'Servidor {SERVIDOR_JIRA} acessado com sucesso!')
except Exception as ex:
  print(f'Erro ao acessar o Jira: {str(ex)}')

Servidor https://issues.apache.org/jira acessado com sucesso!


## Sample 1

Exemplo de como usar a lib jira do Python

In [36]:
# Search for open issues in the Cassandra project
issues = jira.search_issues('project = CASSANDRA and status=resolved',maxResults=10)

for issue in issues:
    print(f'Key: {issue.key}, Summary: {issue.fields.summary}, Status: {issue.fields.status}')

Key: CASSANDRA-18846, Summary: CancellationException when shutting down the 3.11.16 node, Status: Resolved
Key: CASSANDRA-18834, Summary: Testing Pubsub/QBot, Status: Resolved
Key: CASSANDRA-18833, Summary: Testing Pubsub/QBot, Status: Resolved
Key: CASSANDRA-18832, Summary: Add 5.0-alpha1 to cassandra-dtest upgrade_manifest.py, Status: Resolved
Key: CASSANDRA-18829, Summary: Fix flaky test: org.apache.cassandra.index.sai.disk.SelectiveIntersectionTest.tracingIsCorrectlyReported, Status: Resolved
Key: CASSANDRA-18828, Summary: Fix dtest: TestTopology.test_resumable_decommission, Status: Resolved
Key: CASSANDRA-18826, Summary: BLOG - Cassandra Contributor July Meeting Replay, Status: Resolved
Key: CASSANDRA-18823, Summary: Cleanup behaviour during node decommission caused missing replica, Status: Resolved
Key: CASSANDRA-18821, Summary: org.apache.cassandra.distributed.test.guardrails.GuardrailDiskUsageTest failed with Authentication error on host /127.0.0.1:9042: Provided username cassa

In [116]:
issue_CASSANDRA_18647 = jira.issue("CASSANDRA-18647")
print(f'Key: {issue_CASSANDRA_18647.key}, Summary: {issue_CASSANDRA_18647.fields.summary}, Status: {issue_CASSANDRA_18647.fields.status}')
print(f'Date resolved: {issue_CASSANDRA_18647.fields.created}')
print(f'Type: {issue_CASSANDRA_18647.fields.issuetype}')
print(f'Priority: {issue_CASSANDRA_18647.fields.priority}')
print(f'Time spent: {issue_CASSANDRA_18647.fields.timespent} ')
print(f'Description: {issue_CASSANDRA_18647.fields.description}')

Key: CASSANDRA-18647, Summary: CASTing a float to decimal adds wrong digits, Status: Resolved
Date resolved: 2023-07-04T21:27:05.217+0000
Type: Bug
Priority: Normal
Time spent: 600 
Description: If I create a table with a *float* (32-bit) column, and cast it to the *decimal* type, the casting wrongly passes through the double (64-bit) type and picks up extra, wrong, digits. For example, if we have a column e of type "float", and run

INSERT INTO tbl (p, e) VALUES (1, 5.2)

SELECT CAST(e AS decimal) FROM tbl WHERE p=1

The result is the "decimal" value 5.199999809265137, with all those extra wrong digits. It would have been better to get back the decimal value 5.2, with only two significant digits.

It appears that this happens because Cassandra's implementation first converts the 32-bit float into a 64-bit double, and only then converts that - with all the silly extra digits it picked up in the first conversion - into a "decimal" value.

Contrast this with CAST(e AS text) which works c

In [114]:
import datetime

def create_interval_dates(date1, date2, interval=90):
  list_interval_dates = []
  start_date = datetime.datetime(date1[0], date1[1], date1[2])
  end_date = datetime.datetime(date2[0], date2[1], date2[2])
  original_end_date = end_date

  # Calculate the number of days between the start date and the end date.
  days_between_dates = (end_date - start_date).days

  # Create a list of dates that are split by 90 days (default).
  dates = []
  for i in range(0, days_between_dates, interval):
    start_date = start_date + datetime.timedelta(days=i)
    end_date = start_date + datetime.timedelta(days=90)
    if end_date < original_end_date:
      dates.append((start_date, end_date))
    else:
      print(end_date, original_end_date)
      diferenca_dias =  original_end_date - end_date
      print(diferenca_dias.days)
      nova_data = end_date + datetime.timedelta(days=diferenca_dias.days)
      print(nova_data)

  for each in dates:
    date1 = each[0]
    # Convert the date to a string in the format "YYYY/MM/DD".
    str_date1 = date1.strftime("%Y/%m/%d")
    date2 = each[1]
    str_date2 = date2.strftime("%Y/%m/%d")
    elemento = str_date1, str_date2
    list_interval_dates.append(elemento)

  return list_interval_dates

In [115]:
create_interval_dates(date1=(2015,10,3), date2=(2021,6,25), interval=90)

2022-11-25 00:00:00 2021-06-25 00:00:00
-518
2021-06-25 00:00:00
2024-11-14 00:00:00 2021-06-25 00:00:00
-1238
2021-06-25 00:00:00
2027-02-02 00:00:00 2021-06-25 00:00:00
-2048
2021-06-25 00:00:00
2029-07-21 00:00:00 2021-06-25 00:00:00
-2948
2021-06-25 00:00:00
2032-04-06 00:00:00 2021-06-25 00:00:00
-3938
2021-06-25 00:00:00
2035-03-22 00:00:00 2021-06-25 00:00:00
-5018
2021-06-25 00:00:00
2038-06-04 00:00:00 2021-06-25 00:00:00
-6188
2021-06-25 00:00:00
2041-11-15 00:00:00 2021-06-25 00:00:00
-7448
2021-06-25 00:00:00
2045-07-27 00:00:00 2021-06-25 00:00:00
-8798
2021-06-25 00:00:00
2049-07-06 00:00:00 2021-06-25 00:00:00
-10238
2021-06-25 00:00:00
2053-09-13 00:00:00 2021-06-25 00:00:00
-11768
2021-06-25 00:00:00
2058-02-19 00:00:00 2021-06-25 00:00:00
-13388
2021-06-25 00:00:00
2062-10-26 00:00:00 2021-06-25 00:00:00
-15098
2021-06-25 00:00:00
2067-09-30 00:00:00 2021-06-25 00:00:00
-16898
2021-06-25 00:00:00
2072-12-02 00:00:00 2021-06-25 00:00:00
-18788
2021-06-25 00:00:00
2078-

[('2015/10/03', '2016/01/01'),
 ('2016/01/01', '2016/03/31'),
 ('2016/06/29', '2016/09/27'),
 ('2017/03/26', '2017/06/24'),
 ('2018/03/21', '2018/06/19'),
 ('2019/06/14', '2019/09/12'),
 ('2020/12/05', '2021/03/05')]

## Get all issues in range of commits of Critical Files

In [38]:
print('Aguarde...')
t1 = datetime.datetime.now()
print(t1)

issues_by_date_2020_2021_1 = jira.search_issues('project=CASSANDRA and created>="2021-03-01"and created<="2021-06-25"',maxResults=1000)
print("2020/2021/1:", len(issues_by_date_2020_2021_1))

issues_by_date_2020_2021_2 = jira.search_issues('project=CASSANDRA and created>="2020-01-01"and created<"2020-03-01"',maxResults=1000)
print("2020/2021/2:", len(issues_by_date_2020_2021_2))

issues_by_date_2019_2020_1 = jira.search_issues('project=CASSANDRA and created>="2019-06-01"and created<"2020-01-01"',maxResults=1000)
print("2019/2020/1:",len(issues_by_date_2019_2020_1))

issues_by_date_2019_2020_2 = jira.search_issues('project=CASSANDRA and created>="2019-01-01"and created<"2019-06-01"',maxResults=1000)
print("2019/2020/2:",len(issues_by_date_2019_2020_2))

issues_by_date_2018_2019 = jira.search_issues('project=CASSANDRA and created>="2018-01-01"and created<"2019-01-01"',maxResults=1000)
print("2018/2019:",len(issues_by_date_2018_2019))

issues_by_date_2017_2018_1 = jira.search_issues('project=CASSANDRA and created>="2017-06-01"and created<"2018-01-01"',maxResults=1000)
print("2017/2018:",len(issues_by_date_2017_2018_1))

issues_by_date_2017_2018_2 = jira.search_issues('project=CASSANDRA and created>="2017-01-01"and created<"2017-06-01"',maxResults=1000)
print("2017-2:",len(issues_by_date_2017_2018_2))

issues_by_date_2016_2017_1_1 = jira.search_issues('project=CASSANDRA and created>="2016-09-01"and created<"2017-01-01"',maxResults=1000)
print("2016/2017/1:",len(issues_by_date_2016_2017_1_1))

issues_by_date_2016_2017_1_2 = jira.search_issues('project=CASSANDRA and created>="2016-06-01"and created<"2016-09-01"',maxResults=1000)
print("2016/2017/2:",len(issues_by_date_2016_2017_1_2))

issues_by_date_2016_2017_2 = jira.search_issues('project=CASSANDRA and created>="2016-01-01"and created<"2016-06-01"',maxResults=1000)
print("2016.1:",len(issues_by_date_2016_2017_2))

issues_by_date_2015_2016_1 = jira.search_issues('project=CASSANDRA and created>="2015-10-03"and created<"2016-01-01"',maxResults=1000)
print("2015/2016/1:", len(issues_by_date_2015_2016_1))

issues_by_date_2015_2016_2 = jira.search_issues('project=CASSANDRA and created>="2015-08-03"and created<"2015-10-03"',maxResults=1000)
print("2015/2016/2:", len(issues_by_date_2015_2016_2))


t2 = datetime.datetime.now()
print(t2)
print(f'Tempo da consulta: {t2-t1}')

Aguarde...
2023-09-13 15:11:08.580969
2020/2021/1: 292
2020/2021/2: 144
2019/2020/1: 317
2019/2020/2: 192
2018/2019: 799
2017/2018: 574
2017-2: 480
2016/2017/1: 501
2016/2017/2: 650
2016.1: 934
2015/2016/1: 518
2015/2016/2: 485
2023-09-13 15:12:21.320106
Tempo da consulta: 0:01:12.739137


In [39]:
issues_by_date_2015_2021 =  issues_by_date_2020_2021_1 + issues_by_date_2020_2021_2 + issues_by_date_2019_2020_1 + issues_by_date_2019_2020_2 + issues_by_date_2018_2019 + issues_by_date_2017_2018_1 + issues_by_date_2017_2018_2 + issues_by_date_2016_2017_1_1 + issues_by_date_2016_2017_1_2 + issues_by_date_2016_2017_2 + issues_by_date_2015_2016_1 + issues_by_date_2015_2016_2
len(issues_by_date_2015_2021)

5886

In [204]:
dict_issues_2015_2021 = {}

list_key, list_summary, list_status, list_description, list_priority, list_timeestimate, list_timespent, list_issuetype = list(), list(), list(), list(), list(), list(), list(), list()

i = 1
for issue in issues_by_date_2015_2021:
    print(f'{i}, Key: {issue.key}, Summary: {issue.fields.summary}, Status: {issue.fields.status}, Priority: {issue.fields.priority}, Timespent: {issue.fields.timespent}, Time estimate: {issue.fields.timeestimate}')
    list_key.append(issue.key)
    list_summary.append(issue.fields.summary)
    list_status.append(issue.fields.status)
    list_description.append(issue.fields.description)
    list_priority.append(issue.fields.priority)
    list_timespent.append(issue.fields.timespent)
    list_timeestimate.append(issue.fields.timeestimate)
    list_issuetype.append(issue.fields.issuetype)
    i += 1

dict_issues_2015_2021['key'] = list_key
dict_issues_2015_2021['issue_type'] = list_issuetype
dict_issues_2015_2021['summary'] = list_summary
dict_issues_2015_2021['status'] = list_status
dict_issues_2015_2021['description'] = list_description
dict_issues_2015_2021['priority'] = list_priority
dict_issues_2015_2021['timespent'] = list_timespent
dict_issues_2015_2021['timeestimate'] = list_timeestimate

A saída de streaming foi truncada nas últimas 5000 linhas.
887, Key: CASSANDRA-15006, Summary: Possible java.nio.DirectByteBuffer leak, Status: Open, Priority: Normal, Timespent: None, Time estimate: None
888, Key: CASSANDRA-15005, Summary: Configurable whilelist for UDFs, Status: Open, Priority: Low, Timespent: None, Time estimate: None
889, Key: CASSANDRA-15004, Summary: Anti-compaction briefly corrupts sstable state for reads, Status: Resolved, Priority: Urgent, Timespent: None, Time estimate: None
890, Key: CASSANDRA-15002, Summary: Avoid leaking threads when anticompaction fails, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
891, Key: CASSANDRA-15001, Summary: counter not accurate, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
892, Key: CASSANDRA-15000, Summary: Invalidate counter caches on `nodetool import`, Status: Open, Priority: Normal, Timespent: None, Time estimate: None
893, Key: CASSANDRA-14999, Summary: Incorrect fallb

In [221]:
df_cassandra_issues_2015_2021 = pd.DataFrame(dict_issues_2015_2021)
df_cassandra_issues_2015_2021

,key,issue_type,summary,status,description,priority,timespent,timeestimate
0,CASSANDRA-16763,Task,Create Cassandra documentation content for new...,Resolved,We need create the content (asciidoc) to rende...,High,16800.0,0.0
1,CASSANDRA-16762,Task,Create content and UI components for new website,Resolved,We need create the content (asciidoc) and UI c...,High,NaN,NaN
2,CASSANDRA-16761,Epic,New Cassandra Website and Documentation,Resolved,This epic captures the work associated with th...,High,NaN,NaN
3,CASSANDRA-16760,Improvement,JMXTimer exposes attributes in inconsistent ti...,Resolved,JMXTimer objects are constructed with a durati...,Normal,2400.0,0.0
4,CASSANDRA-16759,Bug,Avoid memoizing the wrong min cluster version ...,Resolved,CASSANDRA-16525 avoids trying to calculate the...,Normal,NaN,NaN
...,...,...,...,...,...,...,...,...
5881,CASSANDRA-9959,Bug,Expected bloom filter size should not be an int,Resolved,We cast the expected number of rows in scrub a...,Normal,NaN,NaN
5882,CASSANDRA-9958,Improvement,Add support for the IN operator in secondary i...,Resolved,On a table like:\n{code}\nCREATE Table t (a in...,Low,NaN,NaN
5883,CASSANDRA-9957,Bug,Unable to build Apache Cassandra Under Debian ...,Resolved,Trying to use the tool CCM ( Cassandra Cluster...,Normal,NaN,NaN
5884,CASSANDRA-9956,Bug,Stream failed during a rebuild,Resolved,In an attempt to rebuild a node of a datacente...,Normal,NaN,NaN


In [222]:
lista_key_cassandra_issues_2015_2021 = df_cassandra_issues_2015_2021.key.unique().tolist()
print(lista_key_cassandra_issues_2015_2021)

['CASSANDRA-16763', 'CASSANDRA-16762', 'CASSANDRA-16761', 'CASSANDRA-16760', 'CASSANDRA-16759', 'CASSANDRA-16758', 'CASSANDRA-16757', 'CASSANDRA-16756', 'CASSANDRA-16755', 'CASSANDRA-16754', 'CASSANDRA-16753', 'CASSANDRA-16752', 'CASSANDRA-16751', 'CASSANDRA-16750', 'CASSANDRA-16749', 'CASSANDRA-16748', 'CASSANDRA-16747', 'CASSANDRA-16746', 'CASSANDRA-16745', 'CASSANDRA-16744', 'CASSANDRA-16743', 'CASSANDRA-16742', 'CASSANDRA-16741', 'CASSANDRA-16740', 'CASSANDRA-16739', 'CASSANDRA-16738', 'CASSANDRA-16737', 'CASSANDRA-16736', 'CASSANDRA-16735', 'CASSANDRA-16734', 'CASSANDRA-16733', 'CASSANDRA-16732', 'CASSANDRA-16731', 'CASSANDRA-16730', 'CASSANDRA-16729', 'CASSANDRA-16728', 'CASSANDRA-16727', 'CASSANDRA-16726', 'CASSANDRA-16725', 'CASSANDRA-16724', 'CASSANDRA-16723', 'CASSANDRA-16722', 'CASSANDRA-16721', 'CASSANDRA-16720', 'CASSANDRA-16719', 'CASSANDRA-16718', 'CASSANDRA-16717', 'CASSANDRA-16716', 'CASSANDRA-16715', 'CASSANDRA-16714', 'CASSANDRA-16713', 'CASSANDRA-16712', 'CASSANDRA-

In [223]:
dict_key_commit_valido = {}
list_commits_by_key = []
i = 0
for each_issue in lista_key_cassandra_issues_2015_2021:
  list_commits_by_key = []
  for k,v in dict_message_commits_valido.items():
    if each_issue == v:
      i +=1
      list_commits_by_key.append(k)
  dict_key_commit_valido[each_issue] = list_commits_by_key

print(dict_key_commit_valido)

{'CASSANDRA-16763': [], 'CASSANDRA-16762': [], 'CASSANDRA-16761': [], 'CASSANDRA-16760': [], 'CASSANDRA-16759': [], 'CASSANDRA-16758': [], 'CASSANDRA-16757': [], 'CASSANDRA-16756': [], 'CASSANDRA-16755': [], 'CASSANDRA-16754': [], 'CASSANDRA-16753': [], 'CASSANDRA-16752': [], 'CASSANDRA-16751': [], 'CASSANDRA-16750': [], 'CASSANDRA-16749': [], 'CASSANDRA-16748': [], 'CASSANDRA-16747': [], 'CASSANDRA-16746': [], 'CASSANDRA-16745': [], 'CASSANDRA-16744': [], 'CASSANDRA-16743': [], 'CASSANDRA-16742': [], 'CASSANDRA-16741': [], 'CASSANDRA-16740': [], 'CASSANDRA-16739': [], 'CASSANDRA-16738': [], 'CASSANDRA-16737': [], 'CASSANDRA-16736': ['cb0e4386d8ac2d13e7f594ae3d6cacc0b4246855'], 'CASSANDRA-16735': [], 'CASSANDRA-16734': [], 'CASSANDRA-16733': [], 'CASSANDRA-16732': [], 'CASSANDRA-16731': [], 'CASSANDRA-16730': [], 'CASSANDRA-16729': [], 'CASSANDRA-16728': [], 'CASSANDRA-16727': [], 'CASSANDRA-16726': [], 'CASSANDRA-16725': [], 'CASSANDRA-16724': [], 'CASSANDRA-16723': [], 'CASSANDRA-167

In [224]:
key_commits_by_key = list(dict_key_commit_valido.keys() )
commits_commits_by_key = list(dict_key_commit_valido.values() )

dict_key_commit_valido_temp = {
    'key': key_commits_by_key,
    'commits': commits_commits_by_key
}

In [225]:
df_key_commit_valido = pd.DataFrame(data=dict_key_commit_valido_temp)
df_key_commit_valido

,key,commits
0,CASSANDRA-16763,[]
1,CASSANDRA-16762,[]
2,CASSANDRA-16761,[]
3,CASSANDRA-16760,[]
4,CASSANDRA-16759,[]
...,...,...
5881,CASSANDRA-9959,[]
5882,CASSANDRA-9958,[]
5883,CASSANDRA-9957,[]
5884,CASSANDRA-9956,[]


In [226]:
def get_modified_files_from_commit(key):
  list_files = []
  MESSAGE = 0
  MODIFIED_FILES = 1
  QTD_MODIFIED_FILES = 2
  for each in key:
    if each in dict_commit_modified_files.keys():
      if dict_commit_modified_files[each] is not None:
        list_files.append(dict_commit_modified_files[each][MODIFIED_FILES])
  return list_files

In [227]:
df_key_commit_valido['modified_files'] = df_key_commit_valido['commits'].apply(get_modified_files_from_commit)
df_cassandra_issues_2015_2021['commits'] = df_key_commit_valido['commits']
df_cassandra_issues_2015_2021['modified_files'] = df_key_commit_valido['modified_files']
df_cassandra_issues_2015_2021[['key', 'issue_type', 'commits', 'modified_files']]

,key,issue_type,commits,modified_files
0,CASSANDRA-16763,Task,[],[]
1,CASSANDRA-16762,Task,[],[]
2,CASSANDRA-16761,Epic,[],[]
3,CASSANDRA-16760,Improvement,[],[]
4,CASSANDRA-16759,Bug,[],[]
...,...,...,...,...
5881,CASSANDRA-9959,Bug,[],[]
5882,CASSANDRA-9958,Improvement,[],[]
5883,CASSANDRA-9957,Bug,[],[]
5884,CASSANDRA-9956,Bug,[],[]


In [228]:
df_cassandra_issues_2015_2021

,key,issue_type,summary,status,description,priority,timespent,timeestimate,commits,modified_files
0,CASSANDRA-16763,Task,Create Cassandra documentation content for new...,Resolved,We need create the content (asciidoc) to rende...,High,16800.0,0.0,[],[]
1,CASSANDRA-16762,Task,Create content and UI components for new website,Resolved,We need create the content (asciidoc) and UI c...,High,NaN,NaN,[],[]
2,CASSANDRA-16761,Epic,New Cassandra Website and Documentation,Resolved,This epic captures the work associated with th...,High,NaN,NaN,[],[]
3,CASSANDRA-16760,Improvement,JMXTimer exposes attributes in inconsistent ti...,Resolved,JMXTimer objects are constructed with a durati...,Normal,2400.0,0.0,[],[]
4,CASSANDRA-16759,Bug,Avoid memoizing the wrong min cluster version ...,Resolved,CASSANDRA-16525 avoids trying to calculate the...,Normal,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...
5881,CASSANDRA-9959,Bug,Expected bloom filter size should not be an int,Resolved,We cast the expected number of rows in scrub a...,Normal,NaN,NaN,[],[]
5882,CASSANDRA-9958,Improvement,Add support for the IN operator in secondary i...,Resolved,On a table like:\n{code}\nCREATE Table t (a in...,Low,NaN,NaN,[],[]
5883,CASSANDRA-9957,Bug,Unable to build Apache Cassandra Under Debian ...,Resolved,Trying to use the tool CCM ( Cassandra Cluster...,Normal,NaN,NaN,[],[]
5884,CASSANDRA-9956,Bug,Stream failed during a rebuild,Resolved,In an attempt to rebuild a node of a datacente...,Normal,NaN,NaN,[],[]


In [236]:
df_cassandra_issues_2015_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5886 entries, 0 to 5885
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   key             5886 non-null   object 
 1   issue_type      5886 non-null   object 
 2   summary         5886 non-null   object 
 3   status          5886 non-null   object 
 4   description     5821 non-null   object 
 5   priority        5886 non-null   object 
 6   timespent       306 non-null    float64
 7   timeestimate    339 non-null    float64
 8   commits         5886 non-null   object 
 9   modified_files  5886 non-null   object 
dtypes: float64(2), object(8)
memory usage: 460.0+ KB


## Extract bug issues from Dataframe

Seleciona apenas os bugs issues entre 2015 e 2021 do Jira/Cassandra

3.0.0(2015-08-03) até 3.11.11(2021-06-25)

In [239]:
# Converte para string
df_cassandra_issues_2015_2021['issue_type'] = df_cassandra_issues_2015_2021['issue_type'].astype(str)

In [240]:
df_cassandra_bugs_2015_2021 = df_cassandra_issues_2015_2021.query("issue_type=='Bug'")
df_cassandra_bugs_2015_2021

,key,issue_type,summary,status,description,priority,timespent,timeestimate,commits,modified_files
4,CASSANDRA-16759,Bug,Avoid memoizing the wrong min cluster version ...,Resolved,CASSANDRA-16525 avoids trying to calculate the...,Normal,NaN,NaN,[],[]
5,CASSANDRA-16758,Bug,Flaky ClientResourceLimitsTest,Resolved,Flaky [ClientResourceLimitsTest|https://ci-cas...,Normal,NaN,NaN,[],[]
6,CASSANDRA-16757,Bug,Fix org.apache.cassandra.distributed.upgrade.C...,Resolved,{color:#172b4d}[https://jenkins-cm4.apache.org...,Normal,NaN,NaN,[],[]
7,CASSANDRA-16756,Bug,Fix test testNullClusteringValues - org.apache...,Resolved,https://app.circleci.com/pipelines/github/dcap...,Normal,NaN,NaN,[],[]
8,CASSANDRA-16755,Bug,Spring boot application and AWS Cassandra DB c...,Resolved,"Hi All,\r\n\r\nFacing an issue while starting ...",Normal,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...
5880,CASSANDRA-9960,Bug,UDTs still visible after drop/recreate keyspace,Resolved,When deploying my app from the scratch I run s...,Low,NaN,NaN,[],[]
5881,CASSANDRA-9959,Bug,Expected bloom filter size should not be an int,Resolved,We cast the expected number of rows in scrub a...,Normal,NaN,NaN,[],[]
5883,CASSANDRA-9957,Bug,Unable to build Apache Cassandra Under Debian ...,Resolved,Trying to use the tool CCM ( Cassandra Cluster...,Normal,NaN,NaN,[],[]
5884,CASSANDRA-9956,Bug,Stream failed during a rebuild,Resolved,In an attempt to rebuild a node of a datacente...,Normal,NaN,NaN,[],[]


In [241]:
list_issue_bug_fix_2015_2021 = df_cassandra_bugs_2015_2021.key.to_list()
print('Lista de ID de Issue Bug entre 2015 e 2021')
print(f'{len(list_issue_bug_fix_2015_2021)}, {list_issue_bug_fix_2015_2021}')

Lista de ID de Issue Bug entre 2015 e 2021
3324, ['CASSANDRA-16759', 'CASSANDRA-16758', 'CASSANDRA-16757', 'CASSANDRA-16756', 'CASSANDRA-16755', 'CASSANDRA-16754', 'CASSANDRA-16753', 'CASSANDRA-16752', 'CASSANDRA-16751', 'CASSANDRA-16750', 'CASSANDRA-16748', 'CASSANDRA-16737', 'CASSANDRA-16735', 'CASSANDRA-16733', 'CASSANDRA-16732', 'CASSANDRA-16729', 'CASSANDRA-16728', 'CASSANDRA-16727', 'CASSANDRA-16726', 'CASSANDRA-16724', 'CASSANDRA-16723', 'CASSANDRA-16722', 'CASSANDRA-16721', 'CASSANDRA-16720', 'CASSANDRA-16718', 'CASSANDRA-16716', 'CASSANDRA-16714', 'CASSANDRA-16713', 'CASSANDRA-16712', 'CASSANDRA-16711', 'CASSANDRA-16710', 'CASSANDRA-16708', 'CASSANDRA-16707', 'CASSANDRA-16705', 'CASSANDRA-16704', 'CASSANDRA-16703', 'CASSANDRA-16702', 'CASSANDRA-16700', 'CASSANDRA-16699', 'CASSANDRA-16698', 'CASSANDRA-16694', 'CASSANDRA-16693', 'CASSANDRA-16692', 'CASSANDRA-16691', 'CASSANDRA-16690', 'CASSANDRA-16689', 'CASSANDRA-16688', 'CASSANDRA-16686', 'CASSANDRA-16685', 'CASSANDRA-16684', 

In [242]:
list_commits_bug_fix = []

for k, v in dict_message_commits_valido.items():
  if v in list_issue_bug_fix_2015_2021:
    print(k,v)
    list_commits_bug_fix.append(k)

ebbd516985bc3e2859ae00e63a024b837cb4b429 CASSANDRA-10814
e43883137750d62d8bafb3058d5ee4e322221733 CASSANDRA-10701
4e5867affc6c02b4a84ecee5e82e2ac15c2e3a1c CASSANDRA-10921
f8452838a964c7bfa938277c9fdfc337f9aff886 CASSANDRA-10817
9dafa438a5dcce8674aaa945b1495e70f95d2839 CASSANDRA-10902
df3d0b00b0d9a0725b3f1681e6ce9ffe6d330de4 CASSANDRA-11102
bc3ea66925429b743b672d417700d17e9936b187 CASSANDRA-11102
58821cefb2cc2b1bed0d28533dddea0323ecf487 CASSANDRA-11102
093454205498bad7094cb8fb362d60e95296b092 CASSANDRA-11217
621f446ca4a319adcb50d2479e3fa7121eac6ebd CASSANDRA-11217
965ee082da2b8a3ff5e932e91d261018d9232c75 CASSANDRA-11217
2d420e6c2a362c69d1a66076ac7dd2db37ee9306 CASSANDRA-11217
c947a2c4228816f0e069b2f6fed4b725c9cb710e CASSANDRA-11217
2dab42bce7ce1fe6c6dfee1c6fd8ad5a09734e37 CASSANDRA-10817
e6168672bc421f0d0f90dd45bf3a991be578b3dc CASSANDRA-10902
d200d137823d5b250406bccb35473a8fc2f14faf CASSANDRA-11548
209ebd380b641c4f065e9687186f546f8a50b242 CASSANDRA-11548
d473a2811e97e6f395e23f9c795fa1b

In [243]:
len(list_commits_bug_fix)

78

In [244]:
print(list_commits_bug_fix)

['ebbd516985bc3e2859ae00e63a024b837cb4b429', 'e43883137750d62d8bafb3058d5ee4e322221733', '4e5867affc6c02b4a84ecee5e82e2ac15c2e3a1c', 'f8452838a964c7bfa938277c9fdfc337f9aff886', '9dafa438a5dcce8674aaa945b1495e70f95d2839', 'df3d0b00b0d9a0725b3f1681e6ce9ffe6d330de4', 'bc3ea66925429b743b672d417700d17e9936b187', '58821cefb2cc2b1bed0d28533dddea0323ecf487', '093454205498bad7094cb8fb362d60e95296b092', '621f446ca4a319adcb50d2479e3fa7121eac6ebd', '965ee082da2b8a3ff5e932e91d261018d9232c75', '2d420e6c2a362c69d1a66076ac7dd2db37ee9306', 'c947a2c4228816f0e069b2f6fed4b725c9cb710e', '2dab42bce7ce1fe6c6dfee1c6fd8ad5a09734e37', 'e6168672bc421f0d0f90dd45bf3a991be578b3dc', 'd200d137823d5b250406bccb35473a8fc2f14faf', '209ebd380b641c4f065e9687186f546f8a50b242', 'd473a2811e97e6f395e23f9c795fa1b4b1251788', 'fccded58a6f2ea03dd501c3889e0aa14b6571e16', '00f25401c9a320c0c2ac94fbb1014f49597026fd', '6add3c9acc063005198510e9627ae9e783e54e91', '0e9d6bfe1a182b5ce12e7772ade3433b440aa2c6', 'e89028d34c699916b8c94b52f4cb3e

In [245]:
dict_commit_modified_files_impacted_by_bugs = {}
list_modified_files_impacted_by_bugs = []

for commit in pydriller.Repository("cassandra", only_commits=list_commits_bug_fix).traverse_commits():
    list_modified_files = []
    for m in commit.modified_files:
        list_modified_files.append(m.filename)
        list_modified_files_impacted_by_bugs.append(m.filename)
    dict_commit_modified_files_impacted_by_bugs[commit.hash] = list_modified_files
    print(commit.hash, list_modified_files)

ebbd516985bc3e2859ae00e63a024b837cb4b429 ['DateType.java', 'TimestampType.java', 'TimestampSerializer.java']
e43883137750d62d8bafb3058d5ee4e322221733 ['CQL.textile']
4e5867affc6c02b4a84ecee5e82e2ac15c2e3a1c ['CQL.textile', 'QueryProcessor.java']
f8452838a964c7bfa938277c9fdfc337f9aff886 ['CHANGES.txt', 'Cql.g']
9dafa438a5dcce8674aaa945b1495e70f95d2839 ['CHANGES.txt', 'StartupChecks.java']
df3d0b00b0d9a0725b3f1681e6ce9ffe6d330de4 ['CHANGES.txt', 'MetadataCollector.java', 'TTLExpiryTest.java']
bc3ea66925429b743b672d417700d17e9936b187 ['DeleteTest.java']
58821cefb2cc2b1bed0d28533dddea0323ecf487 ['DeleteTest.java']
093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoa

In [246]:
set_list_modified_files_impacted_by_bugs = set(list_modified_files_impacted_by_bugs)
print(f'{len(set_list_modified_files_impacted_by_bugs)}, {set_list_modified_files_impacted_by_bugs}')

111, {'CommitLogDescriptor.java', 'Cql.g', 'QueryMessage.java', 'YamlConfigurationLoader.java', 'build-rat.xml', 'MigrationManager.java', 'CommitLog.java', 'SSTableWriterTestBase.java', 'LivenessInfo.java', 'View.java', 'QueryProcessor.java', 'LeveledCompactionStrategyTest.java', 'RepairJob.java', 'FBUtilitiesTest.java', 'cassandra', 'ViewUpdateGenerator.java', 'ColumnFilterTest.java', 'TableViews.java', 'SSTableMetadataViewer.java', 'README.md', 'StreamingHistogram.java', 'Tracing.java', 'MessagingService.java', 'StreamingHistogramBench.java', 'RowCacheTest.java', 'idea-iml-file.xml', 'cassandra.spec', 'SSTableCompactingNotification.java', 'FBUtilities.java', 'Row.java', 'SchemaKeyspace.java', 'cassandra-driver-internal-only-3.7.1.post0-19c1603.zip', 'BTreeRow.java', 'ViewManager.java', 'SSTableRewriterTest.java', 'DateType.java', 'build.xml', 'SecondaryIndexTest.java', 'NativeSSTableLoaderClient.java', 'PurgeFunction.java', 'MetadataCollector.java', 'LazilyCompactedRow.java', 'Tracin

## Arquivos críticos identificados pelo ATDCodeAnalyzer

In [247]:
lista_arquivos_criticos_java = []
for each in lista_arquivos_criticos:
  each = each + '.java'
  lista_arquivos_criticos_java.append(each)
lista_arquivos_criticos_java

['StorageService.java',
 'ColumnFamilyStore.java',
 'DatabaseDescriptor.java',
 'CompactionManager.java',
 'StorageProxy.java',
 'SSTableReader.java',
 'Config.java',
 'CassandraDaemon.java',
 'SelectStatement.java',
 'SinglePartitionReadCommand.java',
 'NodeProbe.java',
 'MessagingService.java']

In [248]:
conjunto_lista_arquivos_criticos = set(lista_arquivos_criticos_java)

### Arquivos críticos que aparecem nos bug-fix

In [249]:
classes_criticas_aparece_em_bugfix = conjunto_lista_arquivos_criticos.intersection(set_list_modified_files_impacted_by_bugs)
classes_criticas_aparece_em_bugfix

{'CompactionManager.java',
 'Config.java',
 'DatabaseDescriptor.java',
 'MessagingService.java',
 'SinglePartitionReadCommand.java',
 'StorageService.java'}

In [270]:
my_commits_bugfix = []
for k, v in dict_commit_modified_files_impacted_by_bugs.items():
  for each in classes_criticas_aparece_em_bugfix:
    if each in v:
      print(k,v)
      my_commits_bugfix.append(k)

093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
2d420e6c2a362c69d1a66076ac7dd2db37ee9306 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
2d420e6c2a362c69d1a66076ac7dd2db37ee9306 ['CHANGES.txt', 'Config.java', 'DatabaseDe

In [271]:
set_my_commits_bugfix = set(my_commits_bugfix)

for i in df_cassandra_bugs_2015_2021.index:
  consulta_commits = df_cassandra_bugs_2015_2021['commits'][i]
  if len(consulta_commits) > 0:
    for each_commit in consulta_commits:
      if each_commit in set_my_commits_bugfix:
        print(df_cassandra_bugs_2015_2021['key'][i], df_cassandra_bugs_2015_2021['commits'][i], df_cassandra_bugs_2015_2021['modified_files'][i])

CASSANDRA-16628 ['8fd046f2557e4d50e80986b813bc904dce625e6a'] [['StorageService.java']]
CASSANDRA-16495 ['3edacd632c2f2c360ca1d931c6b4f6fd7326511f'] [['CHANGES.txt', 'ScheduledExecutors.java', 'MigrationCoordinator.java', 'StorageService.java']]
CASSANDRA-13090 ['5725e2c422d21d8efe5ae3bc4389842939553650', '9a80f803c2ec9a4a74cb8a99293dc81ef3dc183d', 'c6462d793b5736b7728c6f188b224dd9d3760bc6', '3748bf7c2a135baab33ccd3b79db5f3fb9132995'] [['CHANGES.txt', 'cassandra.yaml', 'Config.java', 'DatabaseDescriptor.java', 'OutboundTcpConnection.java', 'CoalescingStrategies.java', 'CoalescingStrategiesTest.java'], ['cassandra.yaml'], ['CHANGES.txt'], ['CHANGES.txt']]
CASSANDRA-13004 ['1f54aa424fd8a79089f76951a93560e6bca9d459', '90c5c8bfecd69edb5f859e9ff2424930b1394b49'] [['CHANGES.txt', 'NEWS.txt', 'ReadResponse.java', 'CommitLogDescriptor.java', 'ColumnFilter.java', 'MessagingService.java', 'MigrationManager.java', 'ColumnFilterTest.java'], ['MessageIn.java', 'MessagingService.java']]
CASSANDRA-130